In [267]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

# 输入数据读取与处理
data = pd.read_csv('data/Mobile_Phone_Data_1.csv')
data = data[['timestamp', 'battery_level', 'battery_voltage', 'battery_current', 'battery_power', 'battery_temperature']]
data.columns = ['Time', 'SOC', 'Voltage(V)', 'Current(A)', 'Power(W)', 'Temperature(°C)']

powers = data['Power(W)'].values
times = data['Time'].values
temps = data['Temperature(°C)'].values
data['Current(A)'] = data['Current(A)']   # 转换为A
data.head(10000)

,Time,SOC,Voltage(V),Current(A),Power(W),Temperature(°C)
0,0,0.68,4.110,-1.298339,-5.336173,34.7
1,1,0.68,4.018,-0.891112,-3.580488,34.7
2,2,0.68,4.030,-0.632812,-2.550233,34.7
3,3,0.68,4.031,-0.621581,-2.505593,34.7
4,4,0.68,4.039,-0.452148,-1.826226,34.7
...,...,...,...,...,...,...
9995,9995,0.52,3.847,-0.434570,-1.671791,24.0
9996,9996,0.52,3.847,-0.219238,-0.843409,24.0
9997,9997,0.52,3.844,-0.438964,-1.687378,24.0
9998,9998,0.52,3.844,-0.438964,-1.687378,24.0


In [ ]:
# 求OCV函数
t_ref = 23 # 参考温度，摄氏度
discharging_23c = pd.read_csv('./data/OCV_SOC_23C_discharging.csv')
discharging_45c = pd.read_csv('./data/OCV_SOC_45C_discharging.csv')
sorted_discharging_23c = discharging_23c.sort_values(by='SOC')
sorted_discharging_45c = discharging_45c.sort_values(by='SOC')

# 基准OCV曲线：23摄氏度下放电曲线，最终选用这个
def ocv_ref_23(soc, table):
    # table.sort_values(by='SOC', inplace=True)
    res = np.interp(soc, table['SOC'], table['OCV(mV)'])
    return res # mV
# 温度敏感系数kt(soc)
def kt(soc, table1, table2, T1, T2):
    ocv1 = ocv_ref_23(soc, table1)
    ocv2 = ocv_ref_23(soc, table2)
    return (ocv2 - ocv1) / (T2 - T1) # mV/°C
# 综合OCV模型
def ocv_t(soc, t, soh):
    k_t = kt(soc, sorted_discharging_23c, sorted_discharging_45c, 23, 45)
    return (ocv_ref_23(soc, sorted_discharging_23c) + k_t * (t - t_ref)) * 1000 # V 
def ocv(soc):
    return ocv_ref_23(soc, sorted_discharging_23c) * 1000 # V

$$
\begin{equation}
V(t) = \frac{(OCV - V_{RC}) + \sqrt{(OCV - V_{RC})^2 - 4 \cdot P(t) \cdot R_{in}}}{2}
\end{equation}
$$
$$
\begin{equation}
\begin{cases}
\dfrac{dSOC}{dt}
=
-\dfrac{P(t)}{C_1\bigl(T(t),SOH\bigr)\,V(t)}, \\[10pt]
\dfrac{dV_{RC}}{dt}
=
-\dfrac{1}{R_0 C_0}V_{RC}(t)
+
\dfrac{1}{C_0}\,i(t), \\[10pt]
C_{th}\dfrac{dT}{dt}
=
Q_{gen}(t)
-
h\bigl(T(t)-T_{amb}\bigr),
\end{cases}
\end{equation}$$

In [ ]:
def battery_rk4_step(soc, vrc, p, params, dt, temp):
    """使用 RK4 算法计算一个时间步的状态更新"""
    
    cap_ah = params['capacity']
    r_in = params['r_in']
    r0 = params['r0']
    c0 = params['c0']
    ocv_func = params['ocv_func']

    def derivatives(s, v_r):
        """定义微分方程组 dSOC/dt 和 dVrc/dt"""
        # 1. 再次利用解析解耦公式计算当前 V
        ocv = ocv_func(s)
        b = ocv - v_r
        # 简化处理：若判别式小于0则功率过载，取判别式为0
        v_t = (b + np.sqrt(max(0, b**2 - 4 * p * r_in))) / 2
        i = p / v_t
        
        # 2. 计算两个状态的导数
        ds = -i / (cap_ah * 3600)
        dv = -v_r / (r0 * c0) + i / c0
        return np.array([ds, dv])

    # --- RK4 核心四步探测 ---
    y = np.array([soc, vrc])
    
    k1 = derivatives(y[0], y[1])
    k2 = derivatives(y[0] + 0.5*dt*k1[0], y[1] + 0.5*dt*k1[1])
    k3 = derivatives(y[0] + 0.5*dt*k2[0], y[1] + 0.5*dt*k2[1])
    k4 = derivatives(y[0] + dt*k3[0], y[1] + dt*k3[1])
    
    # 加权平均更新
    y_next = y + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    
    return y_next[0], y_next[1]

In [ ]:
# 模拟运行
n = len(times) # 时间步数
soc = np.zeros(n) # soc序列
soc[0] = data['SOC'].values[0] # 初始SOC
vrc = np.zeros(n) # vrc序列
vrc[0] = 0.0 # 初始vrc
parameters = {
    'capacity': 2.0,  # 电池容量，单位Ah，示例值 TODO 温度影响
    'r_in': 0.45,     # 内阻，示例值 TODO 温度影响
    'r0': 0.22,     # R0参数 TODO 参数估计
    'c0': 23,     # C0参数 TODO 参数估计
    'ocv_func': ocv   # OCV函数
}
for k in range(0, n-1, 1):
    soc[k+1], vrc[k+1] = battery_rk4_step(soc[k], vrc[k], -powers[k], parameters, 60, temps[k])

In [ ]:
# 画图
plt.figure(figsize=(12, 6))
plt.plot(times, soc, label='Simulated SOC', color='blue')
plt.plot(times, data['SOC'].values, label='Measured SOC', color='orange', alpha=0.6)
plt.xlabel('Time(s)')
plt.ylabel('SOC')
plt.legend()
plt.title('Simulated vs Measured SOC Over Time')
plt.show()
soc

In [ ]:
# Vrc曲线
plt.plot(times, vrc, label='Vrc', color='green')
plt.xlabel('Time(s)')
plt.ylabel('Vrc (V)')
plt.show()
vrc[:10]

In [ ]:
plt.plot(times, powers)
plt.xlabel('Time(s)')
plt.ylabel('Power (W)')
plt.show()

In [ ]:
# 计算误差
from sklearn.metrics import mean_absolute_error, mean_squared_error

def check_accuracy(real, pred):
    mae = mean_absolute_error(real, pred)
    rmse = np.sqrt(mean_squared_error(real, pred))
    max_err = np.max(np.abs(real - pred))
    
    print(f"--- 准确度评估 ---")
    print(f"MAE  : {mae:.4f} (平均偏差 {mae*100:.2f}%)")
    print(f"RMSE : {rmse:.4f}")
    print(f"MAXE : {max_err:.4f} (最大偏差 {max_err*100:.2f}%)")
    
    return mae, rmse, max_err

check_accuracy(data['SOC'].values, soc)

gpt版本

In [ ]:
def battery_derivatives(t, state, inputs, params):
    """
    state = [SOC, V_RC, T]
    inputs = dict with keys: P, I, T_amb
    """
    SOC, V_RC, T = state

    P = inputs['P']
    I = inputs['I']
    T_amb = inputs['T_amb']

    # parameters
    R_in = params['r_in']
    R0 = params['r0']
    C0 = params['c0']
    C1 = params['capacity']  # Ah
    C_th = params.get('c_th', 100.0)  # 热容，示例
    h = params.get('h', 5.0)           # 对流系数，示例

    # OCV
    OCV = params['ocv_func'](SOC)

    # Terminal voltage (algebraic equation)
    delta = (OCV - V_RC)**2 - 4 * P * R_in
    delta = max(delta, 1e-6)  # 防止数值炸裂

    V = ((OCV - V_RC) + np.sqrt(delta)) / 2

    # SOC dynamics
    dSOC_dt = - P / (C1 * 3600 * V)   # Ah → As

    # RC voltage dynamics
    dVRC_dt = - V_RC / (R0 * C0) + I / C0

    # Heat generation (简化 Joule loss)
    Q_gen = I**2 * R_in

    # Temperature dynamics
    dT_dt = (Q_gen - h * (T - T_amb)) / C_th

    return np.array([dSOC_dt, dVRC_dt, dT_dt])

def rk4_step(f, t, x, dt, inputs, params):
    k1 = f(t, x, inputs, params)
    k2 = f(t + dt/2, x + dt/2 * k1, inputs, params)
    k3 = f(t + dt/2, x + dt/2 * k2, inputs, params)
    k4 = f(t + dt, x + dt * k3, inputs, params)

    return x + dt / 6 * (k1 + 2*k2 + 2*k3 + k4)

# 模拟主循环

times = data['Time'].values
dt = np.mean(np.diff(times))

n = len(times)
states = np.zeros((n, 3))  # SOC, V_RC, T

# initial conditions
states[0, 0] = data['SOC'].values[0]
states[0, 1] = 0.0
states[0, 2] = data['Temperature(°C)'].values[0]

for k in range(n - 1):
    inputs = {
        'P': data['Power(W)'].values[k],
        'I': data['Current(A)'].values[k],
        'T_amb': data['Temperature(°C)'].values[k]  # 或常数
    }

    states[k+1] = rk4_step(
        battery_derivatives,
        times[k],
        states[k],
        dt,
        inputs,
        parameters
    )

soc_sim = states[:, 0]
vrc_sim = states[:, 1]
temp_sim = states[:, 2]

plt.plot(times, data['SOC'], label='Measured SOC')
plt.plot(times, soc_sim, label='RK4 SOC')
plt.legend()
plt.show()